# 데이터 확인

In [24]:
#훈련데이터 다운로드
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
newsdata=fetch_20newsgroups(subset='train')
print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [4]:
#훈련 샘플 갯수 확인
print (len(newsdata.data), len(newsdata.filenames), len(newsdata.target_names), len(newsdata.target))

11314 11314 20 11314


In [5]:
#훈련 카테고리 종류 확인
print(newsdata.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
#첫번째 샘플의 카테고리 확인
print(newsdata.target[0])

7


In [7]:
#7번째 카테고리의 제목 확인
print(newsdata.target_names[7])

rec.autos


In [20]:
#첫번째 샘플의 내용 확인
print(newsdata.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [25]:
news_df = pd.DataFrame({'news':newsdata.data})

In [29]:
news_df['clean_doc'] = news_df['news'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
news_df['clean_doc'][0]

'from lerxst where thing subject what this nntp posting host organization university maryland college park lines wondering anyone there could enlighten this other door sports looked from late early called bricklin doors were really small addition front bumper separate from rest body this know anyone tellme model name engine specs years production where this made history whatever info have this funky looking please mail thanks brought your neighborhood lerxst'

In [30]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\동211_로봇SI-02\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
print(tokenized_doc[0])

['lerxst', 'thing', 'subject', 'nntp', 'posting', 'host', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'wondering', 'anyone', 'could', 'enlighten', 'door', 'sports', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'made', 'history', 'whatever', 'info', 'funky', 'looking', 'please', 'mail', 'thanks', 'brought', 'neighborhood', 'lerxst']


In [45]:
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
news_df['clean_doc'] = detokenized_doc

In [46]:
news_df['clean_doc'][0]

'lerxst thing subject nntp posting host organization university maryland college park lines wondering anyone could enlighten door sports looked late early called bricklin doors really small addition front bumper separate rest body know anyone tellme model name engine specs years production made history whatever info funky looking please mail thanks brought neighborhood lerxst'

# 나이브 베이즈 분류

In [9]:
#라이브러리 임포트
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [41]:
#DTM(문서단어행렬) 준비
dtmvector = CountVectorizer() #문서 벡터화
X_train_dtm = dtmvector.fit_transform(newsdata.data)
print(X_train_dtm.shape)

(11314, 130107)


In [47]:
#DTM(문서단어행렬) 준비
dtmvector = CountVectorizer() #문서 벡터화
X_train_dtm = dtmvector.fit_transform(news_df['clean_doc'])
print(X_train_dtm.shape)

(11314, 78745)


In [48]:
#TF-IDF(단어빈도-역문서빈도 변환)
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidfv.shape)

(11314, 78745)


In [49]:
#사이킷런을 활용한 베이즈 분류기 생성
mod = MultinomialNB()
#tfidfv 는 입력값, newdata.target는 정답값
mod.fit(tfidfv, newsdata.target)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [50]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True) #테스트 데이터 갖고오기
X_test_dtm = dtmvector.transform(newsdata_test.data) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(X_test_dtm) #DTM을 TF-IDF 행렬로 변환

predicted = mod.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(newsdata_test.target, predicted)) #예측값과 실제값 비교

정확도: 0.8073552841210834
